# Horovod-PyTorch

## Introduction

This recipe shows how to run [Horovod](https://github.com/uber/horovod) distributed training framework for PyTorch using Batch AI.


## Details

- Standard Horovod [pytorch_mnist.py](https://raw.githubusercontent.com/uber/horovod/master/examples/pytorch_mnist.py) example will be used;
- pytorch_mnist.py downloads training data on its own during execution;
- The job will be run on standard tensorflow container batchaitraining/pytorch:0.4.0-cp36-cuda9-cudnn7;
- Horovod framework will be installed in the container using job preparation command line. Note, you can build your own docker image containing tensorflow and horovod instead.
- Standard output of the job will be stored on Azure File Share.

## Instructions

### Install Dependencies and Create Configuration file.
Follow [instructions](/recipes) to install all dependencies and create configuration file.

### Read Configuration and Create Batch AI client

In [ ]:
from __future__ import print_function

from datetime import datetime
import sys

from azure.storage.file import FileService
import azure.mgmt.batchai.models as models

# The BatchAI/utilities folder contains helper functions used by different notebooks
sys.path.append('../../../')
import utilities as utils

cfg = utils.config.Configuration('../../configuration.json')
client = utils.config.create_batchai_client(cfg)

Create Resoruce Group and Batch AI workspace if not exists:

In [ ]:
utils.config.create_resource_group(cfg)
_ = client.workspaces.create(cfg.resource_group, cfg.workspace, cfg.location).result()

## 1. Prepare Training Dataset and Script in Azure Storage

### Create File Share

For this example we will create a new File Share with name `batchaisample` under your storage account. This share will be populated with sample scripts and will contain job's output.

**Note** You don't need to create new file share for every cluster. We are doing this in this sample to simplify resource management for you.

In [ ]:
azure_file_share_name = 'batchaisample'
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.create_share(azure_file_share_name, fail_on_exist=False)
print('Done')

### Deploy Sample Script and Configure the Input Directories

- Download original sample script:

In [ ]:
sample_script_url = 'https://raw.githubusercontent.com/uber/horovod/master/examples/pytorch_mnist.py'
utils.dataset.download_file(sample_script_url, 'pytorch_mnist.py')

- Create a folder in the file share and upload the sample script to it.

In [ ]:
samples_dir = 'horovod_samples'
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.create_directory(
    azure_file_share_name, samples_dir, fail_on_exist=False)
service.create_file_from_path(
    azure_file_share_name, samples_dir, 'pytorch_mnist.py', 'pytorch_mnist.py')

## 2. Create Azure Batch AI Compute Cluster

### Configure Compute Cluster
- For this example we will use a GPU cluster of 2 `STANDARD_NC6` nodes. You can increase the number of nodes by changing `nodes_count` variable;
- We will call the cluster `nc6`;

So, the cluster will have the following parameters:

In [ ]:
nodes_count = 2
cluster_name = 'nc6'

parameters = models.ClusterCreateParameters(
    location=cfg.location,
    vm_size='STANDARD_NC6',
    scale_settings=models.ScaleSettings(
        manual=models.ManualScaleSettings(target_node_count=nodes_count)
    ),
    user_account_settings=models.UserAccountSettings(
        admin_user_name=cfg.admin,
        admin_user_password=cfg.admin_password or None,
        admin_user_ssh_public_key=cfg.admin_ssh_key or None,
    )
)

### Create Compute Cluster

In [ ]:
_ = client.clusters.create(cfg.resource_group, cfg.workspace, cluster_name, parameters).result()

### Monitor Cluster Creation

The `utilities` module contains a helper function allowing to wait for the cluster to become available - all nodes are allocated and finished preparation.

In [ ]:
cluster = client.clusters.get(cfg.resource_group,cfg.workspace, cluster_name)
utils.cluster.print_cluster_status(cluster)

## 3. Run Azure Batch AI Training Job

### Configure Job

- The job will use `batchaitraining/pytorch:0.4.0-cp36-cuda9-cudnn7` container.
- Horovod framework will be installed by job preparation command line;
- Will mount file share at folder with name `afs`. Full path of this folder on a computer node will be `AZ_BATCHAI_JOB_MOUNT_ROOT/afs`;
- Will output standard output and error streams to file share.


In [ ]:
azure_file_share = 'afs'
parameters = models.JobCreateParameters(
     location=cfg.location,
     cluster=models.ResourceId(id=cluster.id),
     node_count=2,
     input_directories=[
        models.InputDirectory(
            id='SCRIPT',
            path='$AZ_BATCHAI_JOB_MOUNT_ROOT/{0}/{1}'.format(azure_file_share, samples_dir))],
     std_out_err_path_prefix='$AZ_BATCHAI_JOB_MOUNT_ROOT/{0}'.format(azure_file_share),
     mount_volumes=models.MountVolumes(
            azure_file_shares=[
                models.AzureFileShareReference(
                    account_name=cfg.storage_account_name,
                    credentials=models.AzureStorageCredentialsInfo(
                        account_key=cfg.storage_account_key),
                    azure_file_url='https://{0}.file.core.windows.net/{1}'.format(
                        cfg.storage_account_name, azure_file_share_name),
                    relative_mount_path=azure_file_share)
            ]
        ),  
     container_settings=models.ContainerSettings(
         image_source_registry=models.ImageSourceRegistry(image='batchaitraining/pytorch:0.4.0-cp36-cuda9-cudnn7')),
     job_preparation=models.JobPreparation(
         command_line='apt update; apt install mpi-default-dev mpi-default-bin -y; pip install horovod'),
     horovod_settings = models.HorovodSettings(
         python_script_file_path='$AZ_BATCHAI_INPUT_SCRIPT/pytorch_mnist.py',
         command_line_args='--batch-size 64 --epochs 5')
)

### Create a training Job and wait for Job completion


In [ ]:
experiment_name = 'horovod_experiment'
experiment = client.experiments.create(cfg.resource_group, cfg.workspace, experiment_name).result()
job_name = datetime.utcnow().strftime('horovod_%m_%d_%Y_%H%M%S')
job = client.jobs.create(cfg.resource_group, cfg.workspace, experiment_name, job_name, parameters).result()
print('Created Job {0} in Experiment {1}'.format(job.name, experiment.name))

### Wait for Job to Finish
The job will start running when the cluster will have enough idle nodes. The following code waits for job to start running printing the cluster state. During job run, the code prints current content of stderr.txt.

**Note** Execution may take several minutes to complete.

In [ ]:
utils.job.wait_for_job_completion(client, cfg.resource_group, cfg.workspace, 
                                  experiment_name, job_name, cluster_name, 'stdouterr', 'stdout.txt')

### List stdout.txt and stderr.txt files for the Job and job preparation command

In [ ]:
files = client.jobs.list_output_files(cfg.resource_group, cfg.workspace, experiment_name, job_name,
                                      models.JobsListOutputFilesOptions(outputdirectoryid='stdouterr')) 
for f in list(files):
    print(f.name, f.download_url or 'directory')

## 4. Clean Up (Optional)

### Delete the Job

In [ ]:
_ = client.jobs.delete(cfg.resource_group, cfg.workspace, experiment_name, job_name)

### Delete the Cluster
When you are finished with the sample and don't want to submit any more jobs you can delete the cluster using the following code.

In [ ]:
_= client.clusters.delete(cfg.resource_group, cfg.workspace, cluster_name)

### Delete File Share
When you are finished with the sample and don't want to submit any more jobs you can delete the file share completely with all files using the following code.

In [ ]:
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.delete_share(azure_file_share_name)